<a href="https://colab.research.google.com/github/am4279/Yelp_Sentiment_Models/blob/main/Yelp_Model_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#Building the Yelp Sentiment Model

## Data Prep

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!tar -xvf  '/content/drive/My Drive/yelp_dataset.tar' #-C 'content/drive/My Drive/yelp_reviews'

In [2]:
import pandas as pd

# DATA_PATH = '/content/drive/MyDrive/yelp_academic_dataset_review.json'
# data = pd.read_json(DATA_PATH, orient='records', lines=True)
# data.shape

### Initializing a Spark kernel to operate on a file this size

In [ ]:
!pip install pyspark
!pip install langdetect

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
import numpy as np

In [ ]:
from langdetect import detect

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [ ]:
!wget -q https://mirror.cogentco.com/pub/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz


In [ ]:
!tar xf spark-3.1.2-bin-hadoop3.2.tgz


In [ ]:
!pip install -q findspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()


In [ ]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [ ]:
review = spark.read.json('/content/drive/MyDrive/yelp_academic_dataset_review.json')

In [ ]:
review.count()

In [ ]:
review.head(10)

[Row(business_id='buF9druCkbuXLX526sGELQ', cool=1, date='2014-10-11 03:34:02', funny=1, review_id='lWC-xP3rd6obsecCYsGZRg', stars=4.0, text="Apparently Prides Osteria had a rough summer as evidenced by the almost empty dining room at 6:30 on a Friday night. However new blood in the kitchen seems to have revitalized the food from other customers recent visits. Waitstaff was warm but unobtrusive. By 8 pm or so when we left the bar was full and the dining room was much more lively than it had been. Perhaps Beverly residents prefer a later seating. \n\nAfter reading the mixed reviews of late I was a little tentative over our choice but luckily there was nothing to worry about in the food department. We started with the fried dough, burrata and prosciutto which were all lovely. Then although they don't offer half portions of pasta we each ordered the entree size and split them. We chose the tagliatelle bolognese and a four cheese filled pasta in a creamy sauce with bacon, asparagus and gran

In [ ]:
review_sample.printSchema()

In [ ]:
review_sample.describe(['stars']).show()

In [ ]:
review_sample.groupby('stars').agg(count('*').alias('record_cnt')).orderBy('stars', ascending=False).show()

In [ ]:
review = review.\
withColumn("sentiment_strength",\
           when(col("stars") == 1, "Strong Negative").\
           when(col("stars") == 2, "Negative").\
           when(col("stars") == 3, "Neutral").\
           when(col("stars") == 4, "Positive").\
           when(col("stars") == 5, "Strong Positive").\
           otherwise("No Sentiment")).\
withColumn("label",\
           when(col("stars") == 1, 0).\
#            when(col("stars") == 2, 0).\
#            when(col("stars") == 3, 2).\
#            when(col("stars") == 4, 1).\
           when(col("stars") == 5, 1).\
           otherwise(3))

In [ ]:
review.groupby('label').agg(count('*').alias('record_cnt')).orderBy('label', ascending=False).show()

+-----+----------+
|label|record_cnt|
+-----+----------+
|    3|   3558071|
|    1|   3814532|
|    0|   1262800|
+-----+----------+



In [ ]:
trainingData = review.filter(review['label'] < 3).select('text', 'label')

In [ ]:
trainingData.limit(5)

text,label
I called WVM on t...,1
This place used t...,0
Probably one of t...,1
I work in the Pru...,1
They NEVER seem t...,0


### Balance sample between classes

In [ ]:
ratio_adjust = 1.0 ## ratio of pos to neg in the df_subsample


counts = trainingData.groupBy('label').count().collect()

if counts[0][1] > counts[1][1]:
    down_class = counts[0][0]
else:
    down_class = counts[1][0]
    

higherBound = counts[0][1]
treshold_to_filter = int(ratio_adjust * float(counts[1][1]) / counts[0][1] * higherBound)
 
randGen = lambda x: np.random.randint(0, higherBound) if x == down_class else -1
 
udfRandGen = udf(randGen, IntegerType())

trainingData = trainingData.withColumn("randIndex", udfRandGen("label"))
trainingData = trainingData.filter(trainingData['randIndex'] < treshold_to_filter).drop('randIndex')
 
print("Distribution of Pos and Neg cases of the down-sampled training data are: \n", \
      trainingData.groupBy("label").count().take(3))

Distribution of Pos and Neg cases of the down-sampled training data are: 
 [Row(label=1, count=1262267), Row(label=0, count=1262800)]


In [ ]:
trainingDataSample = trainingData.sample(False, 0.1, 12345)

In [ ]:
trainingDataSample.count()

251744

In [ ]:
pd_reviews = trainingDataSample.toPandas()
type(pd_reviews)

pandas.core.frame.DataFrame

In [ ]:
# pd_reviews.to_pickle('/temp/yelp_review.pkl')

pd_reviews.to_json('/content/drive/MyDrive/yelp_review.json', orient='records', lines=True)

In [ ]:
pd_reviews.head(5)

,text,label
0,"This place used to be a cool, chill place. Now...",0
1,best pizza on a busy street! my kids love it. ...,1
2,"Seriously, why aren't you at Comfort?\n\nTonig...",1
3,A few months back i went in for a toothache an...,0
4,"Might be helpful if you were looking to buy, b...",0


In [ ]:
pd_reviews.shape

(251784, 2)

#### Saving the review DF with star ratings for multi-class classification

In [ ]:
review_sample = review.sample(False, 0.05, 12345)
review_sample.count()

432079

In [ ]:
pd_stars = review_sample.select('text', 'sentiment_strength').toPandas()

In [ ]:
# pd_stars.to_pickle('/temp/yelp_review_stars.pkl')

pd_stars.to_json('/content/drive/MyDrive/yelp_review_stars.json', orient='records', lines=True)

In [ ]:
pd_stars.head(5)

,text,sentiment_strength
0,This store is pretty good. Not as great as Wal...,Positive
1,"They NEVER seem to get our \norder correct, se...",Strong Negative
2,This place is truly a hidden gem (literally hi...,Positive
3,Incredible donuts. Sometimes you have to go ea...,Strong Positive
4,The ramen here is less than great. It came out...,Negative


In [ ]:
pd_stars.shape

(432079, 2)

#### Detect language and limit to English only

In [ ]:
def lang_detect(x):
    try:
        return detect(x)
    except:
        return '--'

In [ ]:
%time pd_reviews['lang'] = pd_reviews['text'].apply(lambda x: lang_detect(x))

CPU times: user 31min 13s, sys: 32.9 s, total: 31min 46s
Wall time: 31min 50s


In [ ]:
pd_reviews = pd_reviews[pd_reviews['lang']=='en'].reset_index(drop=True)
pd_reviews.shape

(251393, 3)

In [ ]:
pd_reviews.to_json('/content/drive/MyDrive/yelp_train_sentiment.json', orient='records', lines=True)

# Yelp Model Build

In [7]:
#!pip install eli5

In [3]:
import sklearn
import pandas as pd

import wordcloud
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import metrics
import numpy as np

import eli5

## Load Data

In [4]:
yelp_path = '/content/drive/MyDrive/yelp_train_sentiment.json'

yelp = pd.read_json(yelp_path, orient='records', lines=True)
# yelp = pd.read_json(yelp_path, orient='records', lines=True).head(10000)



In [5]:
yelp.shape


(251393, 3)

#### Interpreting results
label = 0 >> Negative Sentiment  
label = 1 >> Positive Sentiment  

In [6]:
pd.set_option('display.max_colwidth', 200)
yelp.head(5)

,text,label,lang
0,"This place used to be a cool, chill place. Now its a bunch of neanderthal bouncers hopped up on steroids acting like the can do whatever they want. There are so many better places in davis square ...",0,en
1,"best pizza on a busy street! my kids love it. i've been recommending to everyone who has a birthday party in that area to order from pizzapalooza and they are not disappointed. the pizza is thick,...",1,en
2,"Seriously, why aren't you at Comfort?\n\nTonight was a special occasion, and we wanted to try something new. Yelp led us to Comfort. So worth it!\n\nThis is a new place, and I guess Watertown i...",1,en
3,A few months back i went in for a toothache and ended up needing a root canal after diagnosis from Dr. Austria. the front desk then comes to me while I'm still in the chair stating it'll cost $140...,0,en
4,"Might be helpful if you were looking to buy, but if you need help with something you currently have then good luck getting any help or response. I walked into this place around 5:30p on a Tuesday...",0,en


In [7]:
# define X and y
X = yelp['text']
y = yelp['label']

print(f"X Shape: {X.shape}")
print(f"y Shape: {y.shape}")

X Shape: (251393,)
y Shape: (251393,)


In [8]:
# split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

print(f"Training records, X_train: {X_train.shape} y_train: {y_train.shape}")
print(f"Testing records, X_test: {X_test.shape} y_test: {y_test.shape}")

Training records, X_train: (188544,) y_train: (188544,)
Testing records, X_test: (62849,) y_test: (62849,)


## Initialize vectorizer


In [9]:
vect = CountVectorizer()

# vect = CountVectorizer(lowercase=False, stop_words='english',
#                                   max_df=0.8, min_df=0.2, max_features=10000, ngram_range=(1,3))

vect = CountVectorizer(lowercase=False, stop_words='english', ngram_range=(1,3))

## Part 4: Building and evaluating BERT Classifier using ktrain package


In [10]:
import warnings
# warnings.filterwarnings('ignore')

import re

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.preprocessing.text import Tokenizer, text_to_word_sequence


In [11]:
#!pip install ktrain

In [12]:
import ktrain
from ktrain import text

#### Check available text classifiers in ktrain

In [13]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


### Prepare source data

In [14]:
sentiment = {0: "Negative", 1: "Positive"}
yelp['sentiment'] = yelp['label'].map(sentiment)

df = yelp[['text', 'sentiment']].rename(columns={'text':'data', 'sentiment':'target'})

df.head(5)

,data,target
0,"This place used to be a cool, chill place. Now its a bunch of neanderthal bouncers hopped up on steroids acting like the can do whatever they want. There are so many better places in davis square ...",Negative
1,"best pizza on a busy street! my kids love it. i've been recommending to everyone who has a birthday party in that area to order from pizzapalooza and they are not disappointed. the pizza is thick,...",Positive
2,"Seriously, why aren't you at Comfort?\n\nTonight was a special occasion, and we wanted to try something new. Yelp led us to Comfort. So worth it!\n\nThis is a new place, and I guess Watertown i...",Positive
3,A few months back i went in for a toothache and ended up needing a root canal after diagnosis from Dr. Austria. the front desk then comes to me while I'm still in the chair stating it'll cost $140...,Negative
4,"Might be helpful if you were looking to buy, but if you need help with something you currently have then good luck getting any help or response. I walked into this place around 5:30p on a Tuesday...",Negative


In [15]:
df.groupby(['target']).count()

,data
target,
Negative,125449
Positive,125944


## STEP 1:  Load and Preprocess the Dataset

Preprocessing the data using the texts_from_array function

In [16]:
maxLen = 200 #each document can be of most <maxLen> words. 0 is used as padding ID.
nGramRange = 1 #size of multi-word phrases to consider
preprocessMode='bert' #Either 'standard' (normal tokenization) or 'bert' tokenization and preprocessing for use with BERT text classification model.
sampleSize = 0.3 #Proportion of training to use for validation

(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_df(train_df = df, 
                                                                    text_column  = 'data', 
                                                                    label_columns = ['target'],
                                                                       val_pct=sampleSize,
                                                                       preprocess_mode=preprocessMode, #text must be preprocessed in a specific way for use with BERT
                                                                       maxlen=maxLen)

['Negative', 'Positive']
        Negative  Positive
64726        0.0       1.0
119754       0.0       1.0
218792       0.0       1.0
76055        0.0       1.0
213697       1.0       0.0
['Negative', 'Positive']
        Negative  Positive
71651        0.0       1.0
227256       0.0       1.0
95149        1.0       0.0
3375         1.0       0.0
179616       0.0       1.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
# maxFeatures = 20000 #num of words to consider in vocabulary
# maxLen = 200 #each document can be of most <maxLen> words. 0 is used as padding ID.
# nGramRange = 1 #size of multi-word phrases to consider
# preprocessMode='standard' #Either 'standard' (normal tokenization) or 'bert' tokenization and preprocessing for use with BERT text classification model.
# sampleSize = 0.3 #Proportion of training to use for validation

# (x_train,  y_train), (x_test, y_test), preproc = text.texts_from_df(train_df = df, 
#                                                                     text_column  = 'data', 
#                                                                     label_columns = ['target'],
#                                                                        val_pct=sampleSize,
#                                                                        preprocess_mode=preprocessMode, #text must be preprocessed in a specific way for use with BERT
#                                                                        maxlen=maxLen, 
#                                                                        max_features=maxFeatures)

## STEP 2:  Loading a pretrained BERT model and wrap it in a `ktrain.Learner` object


In [17]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 200
done.


In [18]:
# model = text.text_classifier('bigru', (x_train, y_train), preproc=preproc)

In [19]:
batchSize = 16 ### Check best size

learner = ktrain.get_learner(model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=batchSize)

In [ ]:
# batchSize = 64 ### Check best size

# learner = ktrain.get_learner(model, 
#                              train_data=(x_train, y_train), 
#                              val_data=(x_test, y_test), 
#                              batch_size=batchSize)

## STEP 3: Train the Model

Recommended in the BERT paper: *5e-5*, *3e-5*, or *2e-5*.

I could have used the ktrain Learning Rate Finder can be used to find a good learning rate by invoking `learner.lr_find()` and `learner.lr_plot()`, prior to training but it would take far more time than it's worth.

The `learner.fit_onecycle` method employs a [1cycle learning rate policy](https://arxiv.org/pdf/1803.09820.pdf).

In [20]:
learningRate = 2e-5
numEpoch = 3

learner.fit_onecycle(learningRate, numEpoch)
#learner.autofit(learningRate, numEpoch)
ktrain.get_predictor(model, preproc).save('/content/drive/MyDrive/bert_yelp_predictor1')



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
10999/10999 [==============================] - 2960s 267ms/step - loss: 0.0575 - accuracy: 0.9800 - val_loss: 0.0405 - val_accuracy: 0.9862
Epoch 2/3
10999/10999 [==============================] - 2937s 267ms/step - loss: 0.0296 - accuracy: 0.9901 - val_loss: 0.0296 - val_accuracy: 0.9904
Epoch 3/3
10999/10999 [==============================] - 2938s 267ms/step - loss: 0.0085 - accuracy: 0.9975 - val_loss: 0.0357 - val_accuracy: 0.9908


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
# # reload Predictor and extract model
# model = ktrain.load_predictor('/content/drive/MyDrive/bert_yelp_predictor').model

# # re-instantiate Learner and continue training
# learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=batchSize)
# #learner.fit_onecycle(learningRate, numEpoch)
# learner.autofit(learningRate, numEpoch)
# ktrain.get_predictor(model, preproc).save('/content/drive/MyDrive/bert_yelp_predictor')

In [ ]:
# learningRate = 0.001
# numEpoch = 3

# # learner.fit_onecycle(learningRate, numEpoch)
# learner.autofit(learningRate, numEpoch)

In [23]:
learner.validate(val_data=(x_test, y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     37691
           1       0.99      0.99      0.99     37727

    accuracy                           0.99     75418
   macro avg       0.99      0.99      0.99     75418
weighted avg       0.99      0.99      0.99     75418



array([[37427,   264],
       [  428, 37299]])